<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/mass_Git_DL_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[tf_gan_tpus](https://github.com/bxck75/gan/blob/master/tensorflow_gan/examples/colab_notebooks/tfgan_on_tpus.ipynb)

[tf_gan](https://github.com/bxck75/gan/blob/master/tensorflow_gan/examples/colab_notebooks/tfgan_tutorial.ipynb)

In [0]:
!rm -r /content/sample_data

#imports
import os,sys
import fnmatch
# list of reps to install

reps=[
#         'EIGENREPS',

#         'bxck75/datasets',
#         'EIGENREPS',
#         'tjwei/Flappy-Turtle.',
#         'tjwei/fonttools',
#         'tjwei/blender3d_import_psk_psa',
#         'lllyasviel/sketchKeras',
#         'Mckinsey666/Anime-Face-Dataset',
#         'chenyuntc/pytorch-book',
#         'lllyasviel/style2paints',
#         'llSourcell/GANS-for-style-transfer',
#         'opencv/open_model_zoo',
#         'hindupuravinash/the-gan-zoo',
#         'corenel/GAN-Zoo',
#         'eriklindernoren/Keras-GAN',
#         'junyanz/CycleGAN',
#         'junyanz/pytorch-CycleGAN-and-pix2pix',
#         'junyanz/iGAN', #----> !wget http://efrosgans.eecs.berkeley.edu/iGAN/datasets/church_64.zip <----dataset 	outdoor_128.zip 	handbag_128.zip !!!
#         'martinarjovsky/WassersteinGAN',
#         'shaoanlu/faceswap-GAN',
#         'LantaoYu/SeqGAN',
#         'tjwei/GANotebooks',
#         'adeshpande3/Tensorflow-Programs-and-Tutorials',
#         'adeshpande3/Generative-Adversarial-Networks',
#         'diegoalejogm/gans',
#         'osh/KerasGAN',
#         'tensorflow/gan',
#         'r9y9/gantts',
#         'jayleicn/animeGAN',
#         'jayleicn/ImageNet-Training',
#         'Zardinality/WGAN-tensorflow',
#         'timsainb/Tensorflow-MultiGPU-VAE-GAN',
#         'Larox/python-moviepy-meetup',
        'tjwei/keras-yolo3',
        'tensorflow/models',
        'tensorflow/datasets',
]
print(reps.sort())

# Gitgo class
class GitGo():
    
  def __init__(self,rep,chdir=True,path='/content/'):
    self.rep = rep
    self.chdir = chdir
    self.path = path
    !mkdir -p {self.path}
    if self.rep == 'help':
        self.help()
    self.go() 
      
  def help(self):
    print("* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo('<rep owner>/<rep name>', <True/False>, <root path>)\
          ")
    sys.exit()
    
  def go(self):
    self.rep=self.rep.split('/')
    # change folder check
    if self.chdir ==True:
        #Switch to path
        os.chdir(self.path)
    # pull get the git rep
    os.system('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git')
    # Set the return value for rep rootpath
    self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + self.PATH)

  def __repr__(self):
    return self.PATH


def get_other_reps(reps):
    GitUsers=[]
    for u in reps:    
        username=u.split('/')[0]
        GitUsers.append(username)
        GHUSER=username
        !curl 'https://api.github.com/users/{GHUSER}/repos?per_page=100' | grep -o 'git@[^"]*' 

    return list(set(GitUsers))

def recursive_glob(treeroot, pattern):
    results = []
    for base, dirs, files in os.walk(treeroot):
        goodfiles = fnmatch.filter(files, pattern)
        results.extend(os.path.join(base, f) for f in goodfiles)
    return results

mylist = []
# install all reps in the list
for rep in reps:
    name=rep.split('.')[0]
    print(name)
    G=GitGo(name,path='/content/installed_resp')
    R=get_other_reps([name])
#     print(R)
    mylist.append([R[0],name])
    
# mylist = get_other_reps(reps)
print(mylist)

recursive_glob('/content/','requirements*.txt')

In [0]:
%cd /content/installed_resp/keras-yolo3/model_data
# !wget https://pjreddie.com/media/files/darknet53.conv.74

!wget https://pjreddie.com/media/files/yolov3.weights
%cd /content/installed_resp/keras-yolo3
!python convert.py -w yolov3.cfg /content/installed_resp/keras-yolo3/model_data/yolov3.weights /content/installed_resp/keras-yolo3/model_data/yolo.h5

In [0]:
# !python convert.py -w darknet53.cfg /content/installed_resp/keras-yolo3/model_data/darknet53.conv.74 /content/installed_resp/keras-yolo3/model_data/darknet53_weights.h5

In [0]:
# !wget 'https://rockstarpresentation.com/wp-content/uploads/2019/02/crowd.jpg'

In [0]:
!python train_yolov3.py

In [0]:
%%writefile /content/installed_resp/keras-yolo3/yolo3/model.py

"""YOLO_v3 Model Defined in Keras."""

from functools import wraps

import numpy as np
import tensorflow as tf
tf.control_flow_ops = tf
from keras import backend as K
from keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2

from yolo3.utils import compose


@wraps(Conv2D)
def DarknetConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)}
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    darknet_conv_kwargs.update(kwargs)
    return Conv2D(*args, **darknet_conv_kwargs)

def DarknetConv2D_BN_Leaky(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1))

def resblock_body(x, num_filters, num_blocks):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # Darknet uses left and top padding instead of 'same' mode
    x = ZeroPadding2D(((1,0),(1,0)))(x)
    x = DarknetConv2D_BN_Leaky(num_filters, (3,3), strides=(2,2))(x)
    for i in range(num_blocks):
        y = compose(
                DarknetConv2D_BN_Leaky(num_filters//2, (1,1)),
                DarknetConv2D_BN_Leaky(num_filters, (3,3)))(x)
        x = Add()([x,y])
    return x

def darknet_body(x):
    '''Darknent body having 52 Convolution2D layers'''
    x = DarknetConv2D_BN_Leaky(32, (3,3))(x)
    x = resblock_body(x, 64, 1)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    x = resblock_body(x, 512, 8)
    x = resblock_body(x, 1024, 4)
    return x

def make_last_layers(x, num_filters, out_filters):
    '''6 Conv2D_BN_Leaky layers followed by a Conv2D_linear layer'''
    x = compose(
            DarknetConv2D_BN_Leaky(num_filters, (1,1)),
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D_BN_Leaky(num_filters, (1,1)),
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D_BN_Leaky(num_filters, (1,1)))(x)
    y = compose(
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D(out_filters, (1,1)))(x)
    return x, y


def yolo_body(inputs, num_anchors, num_classes):
    """Create YOLO_V3 model CNN body in Keras."""
    darknet = Model(inputs, darknet_body(inputs))
    x, y1 = make_last_layers(darknet.output, 512, num_anchors*(num_classes+5))

    x = compose(
            DarknetConv2D_BN_Leaky(256, (1,1)),
            UpSampling2D(2))(x)
    x = Concatenate()([x,darknet.layers[152].output])
    x, y2 = make_last_layers(x, 256, num_anchors*(num_classes+5))

    x = compose(
            DarknetConv2D_BN_Leaky(128, (1,1)),
            UpSampling2D(2))(x)
    x = Concatenate()([x,darknet.layers[92].output])
    x, y3 = make_last_layers(x, 128, num_anchors*(num_classes+5))

    return Model(inputs, [y1,y2,y3])

def tiny_yolo_body(inputs, num_anchors, num_classes):
    '''Create Tiny YOLO_v3 model CNN body in keras.'''
    x1 = compose(
            DarknetConv2D_BN_Leaky(16, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(32, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(64, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(128, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(256, (3,3)))(inputs)
    x2 = compose(
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(512, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'),
            DarknetConv2D_BN_Leaky(1024, (3,3)),
            DarknetConv2D_BN_Leaky(256, (1,1)))(x1)
    y1 = compose(
            DarknetConv2D_BN_Leaky(512, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))(x2)

    x2 = compose(
            DarknetConv2D_BN_Leaky(128, (1,1)),
            UpSampling2D(2))(x2)
    y2 = compose(
            Concatenate(),
            DarknetConv2D_BN_Leaky(256, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))([x2,x1])

    return Model(inputs, [y1,y2])


def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs


def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes


def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores


def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)

    return boxes_, scores_, classes_


def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format

    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer

    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value

    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true


def box_iou(b1, b2):
    '''Return iou tensor

    Parameters
    ----------
    b1: tensor, shape=(i1,...,iN, 4), xywh
    b2: tensor, shape=(j, 4), xywh

    Returns
    -------
    iou: tensor, shape=(i1,...,iN, j)

    '''

    # Expand dim to apply broadcasting.
    b1 = K.expand_dims(b1, -2)
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh/2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # Expand dim to apply broadcasting.
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh/2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou


def yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False):
    '''Return yolo_loss tensor

    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss

    Returns
    -------
    loss: tensor, shape=(1,)

    '''
    num_layers = len(anchors)//3 # default setting
    yolo_outputs = args[:num_layers]
    y_true = args[num_layers:]
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]
    input_shape = K.cast(K.shape(yolo_outputs[0])[1:3] * 32, K.dtype(y_true[0]))
    grid_shapes = [K.cast(K.shape(yolo_outputs[l])[1:3], K.dtype(y_true[0])) for l in range(num_layers)]
    loss = 0
    m = K.shape(yolo_outputs[0])[0] # batch size, tensor
    mf = K.cast(m, K.dtype(yolo_outputs[0]))

    for l in range(num_layers):
        object_mask = y_true[l][..., 4:5]
        true_class_probs = y_true[l][..., 5:]

        grid, raw_pred, pred_xy, pred_wh = yolo_head(yolo_outputs[l],
             anchors[anchor_mask[l]], num_classes, input_shape, calc_loss=True)
        pred_box = K.concatenate([pred_xy, pred_wh])

        # Darknet raw box to calculate loss.
        raw_true_xy = y_true[l][..., :2]*grid_shapes[l][::-1] - grid
        raw_true_wh = K.log(y_true[l][..., 2:4] / anchors[anchor_mask[l]] * input_shape[::-1])
        raw_true_wh = K.switch(object_mask, raw_true_wh, K.zeros_like(raw_true_wh)) # avoid log(0)=-inf
        box_loss_scale = 2 - y_true[l][...,2:3]*y_true[l][...,3:4]

        # Find ignore mask, iterate over each of batch.
        ignore_mask = tf.TensorArray(K.dtype(y_true[0]), size=1, dynamic_size=True)
        object_mask_bool = K.cast(object_mask, 'bool')
        def loop_body(b, ignore_mask):
            true_box = tf.boolean_mask(y_true[l][b,...,0:4], object_mask_bool[b,...,0])
            iou = box_iou(pred_box[b], true_box)
            best_iou = K.max(iou, axis=-1)
            ignore_mask = ignore_mask.write(b, K.cast(best_iou<ignore_thresh, K.dtype(true_box)))
            return b+1, ignore_mask
        _, ignore_mask = tf.control_flow_ops.while_loop(lambda b,*args: b<m, loop_body, [0, ignore_mask])
        ignore_mask = ignore_mask.stack()
        ignore_mask = K.expand_dims(ignore_mask, -1)

        # K.binary_crossentropy is helpful to avoid exp overflow.
        xy_loss = object_mask * box_loss_scale * K.binary_crossentropy(raw_true_xy, raw_pred[...,0:2], from_logits=True)
        wh_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_wh-raw_pred[...,2:4])
        confidence_loss = object_mask * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True)+ \
            (1-object_mask) * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True) * ignore_mask
        class_loss = object_mask * K.binary_crossentropy(true_class_probs, raw_pred[...,5:], from_logits=True)

        xy_loss = K.sum(xy_loss) / mf
        wh_loss = K.sum(wh_loss) / mf
        confidence_loss = K.sum(confidence_loss) / mf
        class_loss = K.sum(class_loss) / mf
        loss += xy_loss + wh_loss + confidence_loss + class_loss
        if print_loss:
            loss = tf.Print(loss, [loss, xy_loss, wh_loss, confidence_loss, class_loss, K.sum(ignore_mask)], message='loss: ')
    return loss


In [0]:
%%writefile /content/installed_resp/keras-yolo3/train_yolov3.py

"""
Retrain the YOLO model for your own dataset.
"""
import tensorflow as tf
tf.control_flow_ops = tf
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data


def _main():
    annotation_path = 'train.txt'
    log_dir = 'logs/000/'
    classes_path = 'model_data/voc_classes.txt'
    anchors_path = 'model_data/yolo_anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)

    input_shape = (416,416) # multiple of 32, hw

    is_tiny_version = len(anchors)==6 # default setting
    if is_tiny_version:
        model = create_tiny_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
    else:
        model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/yolo.h5') # make sure you know what you freeze

    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 32
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint])
        model.save_weights(log_dir + 'trained_weights_stage_1.h5')

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 32 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=100,
            initial_epoch=50,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        model.save_weights(log_dir + 'trained_weights_final.h5')

    # Further training if needed.


def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

if __name__ == '__main__':
    _main()


In [0]:
# !python yolo_v3ideo.py [video_path] [output_path(optional)]

In [0]:
%%writefile yolo_custom.py
#! /usr/bin/env python
# -*- coding: utf-8 -*-
"""
Run a YOLO_v3 style detection model on test images.
"""

import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from keras.utils import multi_gpu_model
gpu_num=1

class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/yolo.h5' # model path or trained weights path
        self.model_path = 'model_data/yolo.h5' # model path or trained weights path
        self.anchors_path = 'model_data/yolo_anchors.txt'
#         self.anchors_path = 'model_data/tiny_yolo_anchors.txt'
#         self.classes_path = 'model_data/voc_classes.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.45
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416) # fixed size or (None, None), hw
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = timer()
        print(end - start)
        return image

    def close_session(self):
        self.sess.close()


def detect_video(yolo, video_path, output_path=""):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    while True:
        return_value, frame = vid.read()
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    yolo.close_session()


def detect_img(yolo):
    while True:
        img = input('Input image filename:')
        try:
            image = Image.open(img)
        except:
            print('Open Error! Try again!')
            continue
        else:
            r_image = yolo.detect_image(image)
            r_image.show()
    yolo.close_session()



if __name__ == '__main__':
    detect_img(YOLO())


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:bxck75/A1_Colabs.git
git@github.com:bxck75/A1_CycleGAN-and-pix2pix_with_colab.git
git@github.com:bxck75/A1_Widgets_cookbook.git
git@github.com:bxck75/admin-finder.git
git@github.com:bxck75/ATSCAN.git
git@github.com:bxck75/audio-deepdream-tf.git
git@github.com:bxck75/audio_video_swapper.git
git@github.com:bxck75/BashBunnie-Linux-evil-script.git
git@github.com:bxck75/bashbunny-payloads.git
git@github.com:bxck75/bashbunny-wifideauth-zombie.git
git@github.com:bxck75/BinGoo.git
git@github.com:bxck75/CamScan-and-GOAhead-exploiter.git
git@github.com:bxck75/CartoonGan-tensorflow.git
git@github.com:bxck75/CartoonGAN-Test-Pytorch-Torch.git
git@github.com:bxck75/Cartoonizer.git
git@github.com:bxck75/censys_io-connector.git
git@github.com:bxck75/chainer.git
git@github.com:bxck75/chat_node.git
git@github.com:bxck75/chimay-red.git
git@github.com:bxck75/ChuClone.git
git@github.com:bxck75/colab_projects.git
git@github.com:bxck75/ComputerVision.git
git@github.com:bxck75/cookbook-2nd-code.git
git@github.com:bxck75/crawler.git
git@github.com:bxck75/cuda9.1cudnn7tf1.7cc3.0python3.5.git
git@github.com:bxck75/DataCamp.git
git@github.com:bxck75/datasets.git
git@github.com:bxck75/DataSetScraper.git
git@github.com:bxck75/dead-simple-grid.git
git@github.com:bxck75/DeeDee_The_Youtube_Tripper.git
git@github.com:bxck75/demo.git
git@github.com:bxck75/documentation.git
git@github.com:bxck75/easy_pix2pix.git
git@github.com:bxck75/esp-idf-template.git
git@github.com:bxck75/eventplanner.git
git@github.com:bxck75/evolving-simple-organisms.git
git@github.com:bxck75/examples.git
git@github.com:bxck75/exploits.git
git@github.com:bxck75/FaceEditor.git
git@github.com:bxck75/Fixed-Symfony-Demo.git
git@github.com:bxck75/fluxion.git
git@github.com:bxck75/Fluxion-1.git
git@github.com:bxck75/footbfirst.git
git@github.com:bxck75/fuzzer-android.git
git@github.com:bxck75/GAIA.git
git@github.com:bxck75/GeneratedNotebookExample.git
git@github.com:bxck75/geoip-api-php.git
git@github.com:bxck75/geoip_mapbox.git
git@github.com:bxck75/git-ftp.git
git@github.com:bxck75/gridism.git
git@github.com:bxck75/HERCULES.git
git@github.com:bxck75/iGAN.git
git@github.com:bxck75/ImageAIDocumentation.git
git@github.com:bxck75/ipysheet.git
git@github.com:bxck75/ipython-popup.git
git@github.com:bxck75/kepler_orrery.git
git@github.com:bxck75/les1.git
git@github.com:bxck75/libgdx.git
git@github.com:bxck75/LSB-Steganography.git
git@github.com:bxck75/lsun.git
git@github.com:bxck75/mandelbulber2.git
git@github.com:bxck75/newcms.git
git@github.com:bxck75/Nodejs-Koajs-API-connect.git
git@github.com:bxck75/normal_hack.git
git@github.com:bxck75/oefenscripts.git
git@github.com:bxck75/PayloadsAllTheThings.git
git@github.com:bxck75/PHP-Crawler.git
git@github.com:bxck75/PHPExcel.git
git@github.com:bxck75/phpWebCralwer.git
git@github.com:bxck75/piss-ant-pix2pix.git
git@github.com:bxck75/pix2code-template.git
git@github.com:bxck75/poc.git
git@github.com:bxck75/PoC-1.git
git@github.com:bxck75/PoC-2.git
git@github.com:bxck75/poc-3.git
git@github.com:bxck75/PocCollect.git
git@github.com:bxck75/PWR_inurlbr.git
git@github.com:bxck75/python-vis-landscape.git
git@github.com:bxck75/python3-project-template.git
git@github.com:bxck75/PythonDataScienceHandbook.git
git@github.com:bxck75/pythreejs.git
git@github.com:bxck75/PyTilemap.git
git@github.com:bxck75/pytorch-CycleGAN-and-pix2pix.git
git@github.com:bxck75/Qbuild.git
git@github.com:bxck75/QtScannerApp.git
git@github.com:bxck75/racegame.git
git@github.com:bxck75/rootbait.git
git@github.com:bxck75/router-exploits.git
git@github.com:bxck75/Saphyra-DDoS.git
git@github.com:bxck75/scanguisym.git
git@github.com:bxck75/SCANNER-INURLBR.git
git@github.com:bxck75/Scraper.git
git@github.com:bxck75/SearchEngineScrapy.git
git@github.com:bxck75/Simple-Grid.git
git@github.com:bxck75/simple_canvas_game.git
git@github.com:bxck75/SlickNav.git
git@github.com:bxck75/slitherembed.git
git@github.com:bxck75/smart-sketch.git
git@github.com:bxck75/space_bar.git
100  541k  100  541k    0     0   631k      0 --:--:-- --:--:-- --:--:--  631k
git@github.com:bxck75/SPADE.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:lllyasviel/chainer-gan-lib.git
git@github.com:lllyasviel/Decompose-Single-Image-Into-Layers.git
git@github.com:lllyasviel/dragon-book-exercise-answers.git
git@github.com:lllyasviel/keras-contrib.git
git@github.com:lllyasviel/lllyasviel.github.io.git
git@github.com:lllyasviel/MangaCraft.git
git@github.com:lllyasviel/progressive_growing_of_gans.git
git@github.com:lllyasviel/sketchKeras.git
git@github.com:lllyasviel/style2paints.git
git@github.com:lllyasviel/TensBlur.git
git@github.com:lllyasviel/the-gan-zoo.git
100 71467  100 71467    0     0   383k      0 --:--:-- --:--:-- --:--:--  383k
git@github.com:lllyasviel/YGOProUnity_V2.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:Mckinsey666/ACGAN-Conditional-Anime-Generation.git
git@github.com:Mckinsey666/Algorithms.git
git@github.com:Mckinsey666/Anime-Face-Dataset.git
git@github.com:Mckinsey666/awesome-image-rectification.git
git@github.com:Mckinsey666/BetterInput.git
git@github.com:Mckinsey666/Breakout-Deep-Q-Learning.git
git@github.com:Mckinsey666/bullet.git
git@github.com:Mckinsey666/chamber-web.git
git@github.com:Mckinsey666/Codeforces-Solutions.git
git@github.com:Mckinsey666/Conditional-GAN-Anime-Generation.git
git@github.com:Mckinsey666/CS-Notes.git
git@github.com:Mckinsey666/Data-Structures.git
git@github.com:Mckinsey666/DCGAN-Random-Anime-Generation.git
git@github.com:Mckinsey666/dsnp_test_cases.git
git@github.com:Mckinsey666/EESA.git
git@github.com:Mckinsey666/ethereal.git
git@github.com:Mckinsey666/Fun-with-MNIST.git
git@github.com:Mckinsey666/GreenJudge-Solutions.git
git@github.com:Mckinsey666/griddy.git
git@github.com:Mckinsey666/Leetcode-Python.git
git@github.com:Mckinsey666/Leetcode-Solutions.git
git@github.com:Mckinsey666/lyricsfinder.git
git@github.com:Mckinsey666/Mahjong-Console-Application.git
git@github.com:Mckinsey666/midterm_foodie.git
git@github.com:Mckinsey666/Pong-Policy-Gradient.git
git@github.com:Mckinsey666/PythonCheatSheet.git
git@github.com:Mckinsey666/Shell-Scripting-CheatSheet.git
git@github.com:Mckinsey666/SNGAN-with-Projection-Discriminator.git
git@github.com:Mckinsey666/Style-Transfer.git
git@github.com:Mckinsey666/UVA-Solutions.git
git@github.com:Mckinsey666/vocabs.git
git@github.com:Mckinsey666/yEETtshen.git
100  184k  100  184k    0     0   571k      0 --:--:-- --:--:-- --:--:--  569k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:chenyuntc/block.git
git@github.com:chenyuntc/caption-tmp.git
git@github.com:chenyuntc/chenyuntc.github.io.git
git@github.com:chenyuntc/chinese-poetry.git
git@github.com:chenyuntc/cloud.git
git@github.com:chenyuntc/crawler.git
git@github.com:chenyuntc/cs231n.git
git@github.com:chenyuntc/deeplearning.git
git@github.com:chenyuntc/DeepLearningForNLPInPytorch.git
git@github.com:chenyuntc/dirbot.git
git@github.com:chenyuntc/DSB2017.git
git@github.com:chenyuntc/dsod.pytorch.git
git@github.com:chenyuntc/examples.git
git@github.com:chenyuntc/flask.git
git@github.com:chenyuntc/horovod.git
git@github.com:chenyuntc/java.git
git@github.com:chenyuntc/keypoint.git
git@github.com:chenyuntc/mmdetection.git
git@github.com:chenyuntc/pose-ae-demo.git
git@github.com:chenyuntc/pox.git
git@github.com:chenyuntc/practical-pytorch.git
git@github.com:chenyuntc/pytorch.git
git@github.com:chenyuntc/pytorch-best-practice.git
git@github.com:chenyuntc/pytorch-book.git
git@github.com:chenyuntc/pytorch-GAN.git
git@github.com:chenyuntc/pytorch-tutorial.git
git@github.com:chenyuntc/PyTorchText.git
git@github.com:chenyuntc/scene-baseline.git
git@github.com:chenyuntc/sexnet.git
git@github.com:chenyuntc/simple-faster-rcnn-pytorch.git
git@github.com:chenyuntc/spark.git
git@github.com:chenyuntc/spark_study.git
git@github.com:chenyuntc/ssd.pytorch.git
git@github.com:chenyuntc/summer.git
git@github.com:chenyuntc/TianchiMedical.git
git@github.com:chenyuntc/tnt.git
git@github.com:chenyuntc/ufldl_tutorial.git
git@github.com:chenyuntc/vision.git
100  213k  100  213k    0     0   546k      0 --:--:-- --:--:-- --:--:--  546k
git@github.com:chenyuntc/WassersteinGAN.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:lllyasviel/chainer-gan-lib.git
git@github.com:lllyasviel/Decompose-Single-Image-Into-Layers.git
git@github.com:lllyasviel/dragon-book-exercise-answers.git
git@github.com:lllyasviel/keras-contrib.git
git@github.com:lllyasviel/lllyasviel.github.io.git
git@github.com:lllyasviel/MangaCraft.git
git@github.com:lllyasviel/progressive_growing_of_gans.git
git@github.com:lllyasviel/sketchKeras.git
git@github.com:lllyasviel/style2paints.git
git@github.com:lllyasviel/TensBlur.git
git@github.com:lllyasviel/the-gan-zoo.git
100 71467  100 71467    0     0   342k      0 --:--:-- --:--:-- --:--:--  343k
git@github.com:lllyasviel/YGOProUnity_V2.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  590k    0   204    0     0    213      0  0:47:16 --:--:--  0:47:16   213git@github.com:llSourcell/100-days-of-code.git
git@github.com:llSourcell/100_Days_of_ML_Code.git
git@github.com:llSourcell/3D_Pose_Estimation.git
git@github.com:llSourcell/7_Research_Directions_Deep_Learning.git
git@github.com:llSourcell/8Puzzle.git
git@github.com:llSourcell/92138.git
git@github.com:llSourcell/A-Guide-to-DeepMinds-StarCraft-AI-Environment.git
git@github.com:llSourcell/actor_critic.git
git@github.com:llSourcell/AFNetworking.git
git@github.com:llSourcell/AI_Artist.git
git@github.com:llSourcell/AI_Composer.git
git@github.com:llSourcell/AI_Dresses_Itself.git
git@github.com:llSourcell/AI_For_Business_Curriculum.git
git@github.com:llSourcell/AI_for_Dating.git
git@github.com:llSourcell/AI_for_Financial_Data.git
git@github.com:llSourcell/AI_for_healthcare.git
git@github.com:llSourcell/AI_For_Music_Composition.git
git@github.com:llSourcell/AI_for_Resumes.git
git@github.com:llSourcell/AI_for_Scheduling.git
git@github.com:llSourcell/AI_for_video_games_demo.git
git@github.com:llSourcell/AI_for_Video_Games_Syllabus.git
git@github.com:llSourcell/AI_Freelancing.git
git@github.com:llSourcell/AI_Humanities.git
git@github.com:llSourcell/AI_in_Finance.git
git@github.com:llSourcell/AI_In_Marketing.git
git@github.com:llSourcell/AI_in_Medicine_Clinical_Imaging_Classification.git
git@github.com:llSourcell/AI_Reader.git
git@github.com:llSourcell/AI_Startup_Prototype.git
git@github.com:llSourcell/AI_Supply_Chain.git
git@github.com:llSourcell/AI_Writer.git
git@github.com:llSourcell/alphago_demo.git
git@github.com:llSourcell/amazon-dsstne.git
git@github.com:llSourcell/Amazon_Training_Video.git
git@github.com:llSourcell/anomaly_detection_for_CERN.git
git@github.com:llSourcell/antivirus_demo.git
git@github.com:llSourcell/An_Introduction_to_GPU_Programming.git
git@github.com:llSourcell/an_intro_to_probabilistic_programming.git
git@github.com:llSourcell/API_Chatbot.git
git@github.com:llSourcell/artistic-style-transfer.git
git@github.com:llSourcell/asteroids_game_bot_LIVE.git
git@github.com:llSourcell/autoencoder_demo.git
git@github.com:llSourcell/autoencoder_explained.git
git@github.com:llSourcell/awesome-public-datasets.git
git@github.com:llSourcell/Azadi.git
git@github.com:llSourcell/Azadi_Complete.git
git@github.com:llSourcell/azure_machine_learning.git
git@github.com:llSourcell/A_guide_to_coreML.git
git@github.com:llSourcell/A_Guide_to_Running_Tensorflow_Models_on_Android.git
git@github.com:llSourcell/backpropagation_explained.git
git@github.com:llSourcell/basic_income.git
git@github.com:llSourcell/bert-as-service.git
git@github.com:llSourcell/Best-Programming-Languages-for-Machine-Learning.git
git@github.com:llSourcell/biaxial-rnn-music-composition.git
git@github.com:llSourcell/bitcoin.git
git@github.com:llSourcell/bitcoin_prediction.git
git@github.com:llSourcell/Bitcoin_Trading_Bot.git
git@github.com:llSourcell/Blabber.git
git@github.com:llSourcell/BlabSay--collab-.git
git@github.com:llSourcell/BlabSayFaye.git
git@github.com:llSourcell/BlabSay_Mobile_App.git
git@github.com:llSourcell/blockchain-python-tutorial.git
git@github.com:llSourcell/blockchain_consensus.git
git@github.com:llSourcell/blockchain_supply_chain.git
git@github.com:llSourcell/block_grasp_generator.git
git@github.com:llSourcell/Boston_Dynamics_Atlas_Explained.git
git@github.com:llSourcell/Brain_Computer_Interface.git
git@github.com:llSourcell/bubbles.git
git@github.com:llSourcell/Build-a-Cybersecurity-Startup.git
git@github.com:llSourcell/Build-a-Retail-Startup.git
git@github.com:llSourcell/Build-an-AI-Startup-with-PyTorch.git
git@github.com:llSourcell/build_a_game_bot_live.git
git@github.com:llSourcell/build_a_neural_net_live.git
git@github.com:llSourcell/CallRoulette-Android.git
git@github.com:llSourcell/CallRoulette-Python.git
git@github.com:llSourcell/capsule_networks.git
git@github.com:llSourcell/car-behavioral-cloning.git
git@github.com:llSourcell/Carbon-Footprint-Multi-Agent-Reinforcement-Learning.git
git@github.com:llSourcell/chat.git
git@github.com:llSourcell/Chatbot-AI.git
git@github.com:llSourcell/chatbot_tutorial.git
git@github.com:llSourcell/ChatterBee.git
git@github.com:llSourcell/citius-invaders.git
git@github.com:llSourcell/clam.git
git@github.com:llSourcell/Classifying_Data_Using_a_Support_Vector_Machine.git
git@github.com:llSourcell/coinhive-stratum-mining-proxy.git
git@github.com:llSourcell/conference.git
git@github.com:llSourcell/Convolutional_neural_network.git
git@github.com:llSourcell/Cryptokitties.git
git@github.com:llSourcell/Crypto_game_tokens.git
git@github.com:llSourcell/crypto_math.git
git@github.com:llSourcell/c_programming_for_machine_learning.git
git@github.com:llSourcell/D3_Data_visualization_live.git
git@github.com:llSourcell/dapp.git
git@github.com:llSourcell/datawallet.git
git@github.com:llSourcell/Data_Science_Interview_Guide.git
git@github.com:llSourcell/Data_Visualization.git
git@github.com:llSourcell/DCRoundSwitch.git
git@github.com:llSourcell/Decentralized_AI.git
git@github.com:llSourcell/Decentralized_Chat.git
100  590k  100  590k    0     0   580k      0  0:00:01  0:00:01 --:--:--  580k
git@github.com:llSourcell/Decentralized_Games.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:opencv/ade.git
git@github.com:opencv/cvat.git
git@github.com:opencv/dldt.git
git@github.com:opencv/gst-video-analytics.git
git@github.com:opencv/jetson-screencasts-sources.git
git@github.com:opencv/opencv.git
git@github.com:opencv/opencv_3rdparty.git
git@github.com:opencv/opencv_attic.git
git@github.com:opencv/opencv_contrib.git
git@github.com:opencv/opencv_extra.git
git@github.com:opencv/opencv_for_ios_book_samples.git
git@github.com:opencv/openvino_training_extensions.git
git@github.com:opencv/open_model_zoo.git
100 79665  100 79665    0     0   342k      0 --:--:-- --:--:-- --:--:--  342k
git@github.com:opencv/training_toolbox_caffe.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:hindupuravinash/android.git
git@github.com:hindupuravinash/android-chat-ui.git
git@github.com:hindupuravinash/androiddev.git
git@github.com:hindupuravinash/awesome-machine-learning.git
git@github.com:hindupuravinash/bricks.git
git@github.com:hindupuravinash/coffeebot.git
git@github.com:hindupuravinash/deep-photo-styletransfer.git
git@github.com:hindupuravinash/facebook-android-sdk.git
git@github.com:hindupuravinash/hindupuravinash.github.io.git
git@github.com:hindupuravinash/LightGBM.git
git@github.com:hindupuravinash/nips2016.git
git@github.com:hindupuravinash/nips2017.git
git@github.com:hindupuravinash/paho.mqtt.android.git
git@github.com:hindupuravinash/paho.mqtt.java.git
git@github.com:hindupuravinash/paper-notes.git
git@github.com:hindupuravinash/parceler.git
git@github.com:hindupuravinash/picasso.git
git@github.com:hindupuravinash/practical1.git
git@github.com:hindupuravinash/PredicateEditor.git
git@github.com:hindupuravinash/PythonDataScienceHandbook.git
git@github.com:hindupuravinash/railtree.git
100  172k  100  172k    0     0   505k      0 --:--:-- --:--:-- --:--:--  504k
git@github.com:hindupuravinash/showtime.git
git@github.com:hindupuravinash/sillyreflections.com.git
git@github.com:hindupuravinash/t-SNE-tutorial.git
git@github.com:hindupuravinash/the-gan-zoo.git
git@github.com:hindupuravinash/the-incredible-pytorch.git
git@github.com:hindupuravinash/WebFundamentals.git
git@github.com:hindupuravinash/WhirlwindTourOfPython.git
git@github.com:hindupuravinash/ygloo-ymagine.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  455k    0   364    0     0    385      0  0:20:12 --:--:--  0:20:12   384git@github.com:corenel/2018sixmgb-spider.git
git@github.com:corenel/algorithm-exercises.git
git@github.com:corenel/autohome-image-spider.git
git@github.com:corenel/bin2img.git
git@github.com:corenel/BoxCars.git
git@github.com:corenel/caffe-dqn-hfo.git
git@github.com:corenel/corenel.github.io.git
git@github.com:corenel/corenel.github.io.bak.git
git@github.com:corenel/course-notes.git
git@github.com:corenel/Cpp-Primer.git
git@github.com:corenel/crypko-spider.git
git@github.com:corenel/CS231n.git
git@github.com:corenel/CV-Papers.git
git@github.com:corenel/darknet.git
git@github.com:corenel/deeplab-public.git
git@github.com:corenel/dldt.git
git@github.com:corenel/dotfile-ubuntu.git
git@github.com:corenel/dotfiles.git
git@github.com:corenel/dotfiles-termux.git
git@github.com:corenel/dqn-hfo.git
git@github.com:corenel/EasyPR.git
git@github.com:corenel/env-setup.git
git@github.com:corenel/fast-gaussian-filter.git
git@github.com:corenel/flambeau.git
git@github.com:corenel/GAN-Zoo.git
git@github.com:corenel/gco_python.git
git@github.com:corenel/git-label-packages.git
git@github.com:corenel/google-play-music-convertor.git
git@github.com:corenel/gpuview.git
git@github.com:corenel/hexo-theme-next.git
git@github.com:corenel/HFO.git
git@github.com:corenel/hugo-theme-even.git
git@github.com:corenel/image-selector.git
git@github.com:corenel/imagetagger.git
git@github.com:corenel/installTensorFlowTX1.git
git@github.com:corenel/Intel-NUC-DSDT-Patch.git
git@github.com:corenel/jetnet-wrapper.git
git@github.com:corenel/LabManage.git
git@github.com:corenel/leetcode.git
git@github.com:corenel/lintcode.git
git@github.com:corenel/makegirlsmoe_web.git
git@github.com:corenel/matlab-socketio.git
git@github.com:corenel/Mess.git
git@github.com:corenel/mqtt-usrcloud-wrapper.git
git@github.com:corenel/MultiAgentAStar.git
git@github.com:corenel/ncsdk.git
git@github.com:corenel/Notes-for-ML.git
git@github.com:corenel/nowcoder.git
git@github.com:corenel/ns-album-exif.git
git@github.com:corenel/NUC8i7BEH-macOS.git
git@github.com:corenel/PRM.git
git@github.com:corenel/pygco.git
git@github.com:corenel/pytorch-adda.git
git@github.com:corenel/pytorch-arda.git
git@github.com:corenel/pytorch-atda.git
git@github.com:corenel/pytorch-docker.git
git@github.com:corenel/pytorch-glow.git
git@github.com:corenel/pytorch-starter-kit.git
git@github.com:corenel/pytorch-yolo2.git
git@github.com:corenel/pytorch-yolo3.git
git@github.com:corenel/QSC-Video-Tutorial.git
git@github.com:corenel/Realistic-Neural-Talking-Head-Models.git
git@github.com:corenel/selective_search_py.git
git@github.com:corenel/silent-screenshot.git
git@github.com:corenel/tensorflow-deeplab-resnet.git
git@github.com:corenel/tf-pose-estimation.git
git@github.com:corenel/torchsharp.git
git@github.com:corenel/torchzoo.git
git@github.com:corenel/Toys.git
git@github.com:corenel/traffic-prediction.git
git@github.com:corenel/TTUR.git
git@github.com:corenel/Udacity-Deep-Learning.git
git@github.com:corenel/vue-lab-manage.git
git@github.com:corenel/walkman-lyrics.git
git@github.com:corenel/walkman-utils.git
git@github.com:corenel/white-balancer.git
git@github.com:corenel/X2Face.git
git@github.com:corenel/XPS9550-macOS.git
git@github.com:corenel/ysvim.git
git@github.com:corenel/yt8m-feature-extractor.git
git@github.com:corenel/YUV2JPG.git
git@github.com:corenel/Z270-HD3-macOS.git
100  455k  100  455k    0     0   451k      0  0:00:01  0:00:01 --:--:--  451k
git@github.com:corenel/ZJUProposal.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:eriklindernoren/Action-Recognition.git
git@github.com:eriklindernoren/awesome-machine-learning.git
git@github.com:eriklindernoren/eriklindernoren.se.git
git@github.com:eriklindernoren/eth-circle.git
git@github.com:eriklindernoren/eth-moodi.git
git@github.com:eriklindernoren/Fast-Neural-Style-Transfer.git
git@github.com:eriklindernoren/GoogleTTS.git
git@github.com:eriklindernoren/HomeAssistant.git
git@github.com:eriklindernoren/Keras-GAN.git
git@github.com:eriklindernoren/MapApp.git
git@github.com:eriklindernoren/ML-From-Scratch.git
git@github.com:eriklindernoren/NapkinML.git
git@github.com:eriklindernoren/Object-Detection.git
git@github.com:eriklindernoren/PFTA-Final-Assignment.git
git@github.com:eriklindernoren/PyTorch-Deep-Dream.git
git@github.com:eriklindernoren/PyTorch-GAN.git
git@github.com:eriklindernoren/PyTorch-YOLOv3.git
git@github.com:eriklindernoren/SemSegGUI.git
git@github.com:eriklindernoren/Sonus-JFugueUI.git
git@github.com:eriklindernoren/SSD-Tensorflow.git
git@github.com:eriklindernoren/TAOP33-Combinatorial-Optimization.git
git@github.com:eriklindernoren/TBMI26-Neural-Networks.git
git@github.com:eriklindernoren/TDDB68-Pintos.git
git@github.com:eriklindernoren/TDDC17-Artificial-Intelligence.git
100  150k  100  150k    0     0   533k      0 --:--:-- --:--:-- --:--:--  534k
git@github.com:eriklindernoren/TDDE07-Bayesian-Learning.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:junyanz/BicycleGAN.git
git@github.com:junyanz/CatPapers.git
git@github.com:junyanz/cnnvisualizer.git
git@github.com:junyanz/CycleGAN.git
git@github.com:junyanz/FaceDemo.git
git@github.com:junyanz/iGAN.git
git@github.com:junyanz/interactive-deep-colorization.git
git@github.com:junyanz/light-field-video.git
git@github.com:junyanz/MCILBoost.git
git@github.com:junyanz/MirrorMirror.git
git@github.com:junyanz/pix2pix.git
git@github.com:junyanz/pytorch-CycleGAN-and-pix2pix.git
git@github.com:junyanz/RealismCNN.git
git@github.com:junyanz/SelectGoodFace.git
100 84453  100 84453    0     0   335k      0 --:--:-- --:--:-- --:--:--  335k
git@github.com:junyanz/VON.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:junyanz/BicycleGAN.git
git@github.com:junyanz/CatPapers.git
git@github.com:junyanz/cnnvisualizer.git
git@github.com:junyanz/CycleGAN.git
git@github.com:junyanz/FaceDemo.git
git@github.com:junyanz/iGAN.git
git@github.com:junyanz/interactive-deep-colorization.git
git@github.com:junyanz/light-field-video.git
git@github.com:junyanz/MCILBoost.git
git@github.com:junyanz/MirrorMirror.git
git@github.com:junyanz/pix2pix.git
git@github.com:junyanz/pytorch-CycleGAN-and-pix2pix.git
git@github.com:junyanz/RealismCNN.git
git@github.com:junyanz/SelectGoodFace.git
100 84453  100 84453    0     0   355k      0 --:--:-- --:--:-- --:--:--  357k
git@github.com:junyanz/VON.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:junyanz/BicycleGAN.git
git@github.com:junyanz/CatPapers.git
git@github.com:junyanz/cnnvisualizer.git
git@github.com:junyanz/CycleGAN.git
git@github.com:junyanz/FaceDemo.git
git@github.com:junyanz/iGAN.git
git@github.com:junyanz/interactive-deep-colorization.git
git@github.com:junyanz/light-field-video.git
git@github.com:junyanz/MCILBoost.git
git@github.com:junyanz/MirrorMirror.git
git@github.com:junyanz/pix2pix.git
git@github.com:junyanz/pytorch-CycleGAN-and-pix2pix.git
git@github.com:junyanz/RealismCNN.git
git@github.com:junyanz/SelectGoodFace.git
100 84453  100 84453    0     0   429k      0 --:--:-- --:--:-- --:--:--  429k
git@github.com:junyanz/VON.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:martinarjovsky/examples.git
git@github.com:martinarjovsky/pytorch.git
git@github.com:martinarjovsky/pytorch-semseg.git
git@github.com:martinarjovsky/Rabin-Miller.git
git@github.com:martinarjovsky/Reversi-AI.git
git@github.com:martinarjovsky/Support-Vector-Machine.git
git@github.com:martinarjovsky/vision.git
100 47842  100 47842    0     0   233k      0 --:--:-- --:--:-- --:--:--  233k
git@github.com:martinarjovsky/WassersteinGAN.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  139k    0   364    0     0    606      0  0:03:55 --:--:--  0:03:55   606git@github.com:shaoanlu/AdamW-and-SGDW.git
git@github.com:shaoanlu/Audio2Guitarist-GAN.git
git@github.com:shaoanlu/Conditional-Analogy-GAN-keras.git
git@github.com:shaoanlu/deeplearning.ai-Convolutional-Neural-Networks.git
git@github.com:shaoanlu/dogs-vs-cats-redux.git
git@github.com:shaoanlu/experiment-with-res2net.git
git@github.com:shaoanlu/experiment-with-swish.git
git@github.com:shaoanlu/expriment-with-focal-loss.git
git@github.com:shaoanlu/face-segmentation-keras.git
git@github.com:shaoanlu/faceswap-GAN.git
git@github.com:shaoanlu/face_toolbox_keras.git
git@github.com:shaoanlu/fast-depth.git
git@github.com:shaoanlu/fasttext_model.git
git@github.com:shaoanlu/fewshot-face-translation-GAN.git
git@github.com:shaoanlu/GazeML-keras.git
git@github.com:shaoanlu/GroupNormalization-keras.git
git@github.com:shaoanlu/miscellaneous-experiments.git
git@github.com:shaoanlu/MUNIT-keras.git
git@github.com:shaoanlu/selu-experiment.git
git@github.com:shaoanlu/shaoanlu.github.io.git
git@github.com:shaoanlu/TextSummarizer.git
git@github.com:shaoanlu/tf_ThinPlateSpline.git
git@github.com:shaoanlu/Udacity-SDCND-Vehicle-Detection.git
100  139k  100  139k    0     0   218k      0 --:--:-- --:--:-- --:--:--  218k
git@github.com:shaoanlu/zoosearch.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:LantaoYu/1M-agents-RL.git
git@github.com:LantaoYu/Cooperative-Training.git
git@github.com:LantaoYu/DeepST.git
git@github.com:LantaoYu/irgan.git
git@github.com:LantaoYu/lantaoyu.github.io.git
git@github.com:LantaoYu/MAgent.git
git@github.com:LantaoYu/MARL-Papers.git
git@github.com:LantaoYu/Residual-Networks.git
git@github.com:LantaoYu/SeqGAN.git
git@github.com:LantaoYu/tinyflow.git
100 60805  100 60805    0     0   288k      0 --:--:-- --:--:-- --:--:--  288k
git@github.com:LantaoYu/v97.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  422k    0   364    0     0    598      0  0:12:03 --:--:--  0:12:03   597git@github.com:tjwei/2048-NN.git
git@github.com:tjwei/acer-stream-minimal-kernel.git
git@github.com:tjwei/alog.git
git@github.com:tjwei/Animation-with-Identical-Statistics.git
git@github.com:tjwei/AsusWebStorage.git
git@github.com:tjwei/awesome-tw-foss.git
git@github.com:tjwei/blender3d_import_psk_psa.git
git@github.com:tjwei/check-io-coins.git
git@github.com:tjwei/checkio-task-template.git
git@github.com:tjwei/ck101fs.git
git@github.com:tjwei/class2016.git
git@github.com:tjwei/codecombat.git
git@github.com:tjwei/ComputerVisionWorkshop.git
git@github.com:tjwei/CrashCourseML.git
git@github.com:tjwei/cython.git
git@github.com:tjwei/cython_vagrant.git
git@github.com:tjwei/DIY_AI.git
git@github.com:tjwei/droplet.git
git@github.com:tjwei/Emudroid-Common.git
git@github.com:tjwei/ETC_math.git
git@github.com:tjwei/EyePython.git
git@github.com:tjwei/fight99.git
git@github.com:tjwei/Flappy-Turtle.git
git@github.com:tjwei/fonttools.git
git@github.com:tjwei/Fractran.git
git@github.com:tjwei/GANotebooks.git
git@github.com:tjwei/GAN_Tutorial.git
git@github.com:tjwei/geekpy.git
git@github.com:tjwei/gh_show.git
git@github.com:tjwei/gulp-compass.git
git@github.com:tjwei/h4-talk-2016-04-21.git
git@github.com:tjwei/HackNTU_Data_2017.git
git@github.com:tjwei/hello.git
git@github.com:tjwei/inumpy.git
git@github.com:tjwei/jedi.git
git@github.com:tjwei/keras-yolo3.git
git@github.com:tjwei/KindleChewing.git
git@github.com:tjwei/LinearAlgebra.git
git@github.com:tjwei/live_reveal.git
git@github.com:tjwei/logic.git
git@github.com:tjwei/math-can-see-dimensions.git
git@github.com:tjwei/mental_poker.git
git@github.com:tjwei/merge-conflicts.git
git@github.com:tjwei/my-first-repo.git
git@github.com:tjwei/my_second_repo.git
git@github.com:tjwei/NCTU_DeepLearning.git
git@github.com:tjwei/NDHU_AM1130.git
git@github.com:tjwei/NDHU_AM_Course_Map.git
git@github.com:tjwei/Neural-Matching.git
git@github.com:tjwei/NeuralNetwork-Jobspace-slides.git
git@github.com:tjwei/numpy_tutorial.git
git@github.com:tjwei/obama_pix2pix.git
git@github.com:tjwei/pcsxjs.git
git@github.com:tjwei/pdfrw.git
git@github.com:tjwei/pix2pix.git
git@github.com:tjwei/PlayVideoData.git
git@github.com:tjwei/play_nsfw.git
git@github.com:tjwei/PyDPTS1.git
git@github.com:tjwei/pypoker-eval.git
git@github.com:tjwei/PythonTutorial.git
git@github.com:tjwei/Quick-Neural-Art-Transfer.git
git@github.com:tjwei/rl.git
git@github.com:tjwei/snes9x-3d.git
git@github.com:tjwei/SNesoid.git
git@github.com:tjwei/spynner.git
git@github.com:tjwei/taishin20190827.git
git@github.com:tjwei/tensorflow.git
10git@github.com:tjwei/tensorflow-tutorial.git
0  4git@github.com:tjwei/test.git
22k git@github.com:tjwei/test1.git
 10git@github.com:tjwei/tf-keras-tutorial.git
0  4git@github.com:tjwei/tf-play.git
22kgit@github.com:tjwei/tjwei.github.io.git
  git@github.com:tjwei/tjw_ipynb.git
  git@github.com:tjwei/tone_of_life.git
0 git@github.com:tjwei/translatePDF.git
   git@github.com:tjwei/WebKitGtkKindleDXG.git
 0   git@github.com:tjwei/word2vec.git
640k      0 --:--:-- --:--:-- --:--:--  639k
git@github.com:tjwei/xnes.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:adeshpande3/Accelerometer-Pogram.git
git@github.com:adeshpande3/adeshpande3.github.io.git
git@github.com:adeshpande3/Chatbot-Flask-Server.git
git@github.com:adeshpande3/Facebook-Messenger-Bot.git
git@github.com:adeshpande3/Generative-Adversarial-Networks.git
git@github.com:adeshpande3/Job-Descriptions-Hacker-Rank-Comp.git
git@github.com:adeshpande3/Kaggle-MNIST.git
git@github.com:adeshpande3/Kaggle-Zillow.git
git@github.com:adeshpande3/KaggleGhosts.git
git@github.com:adeshpande3/KaggleTitanic.git
git@github.com:adeshpande3/LSTM-Sentiment-Analysis.git
git@github.com:adeshpande3/Machine-Learning-Links-And-Lessons-Learned.git
git@github.com:adeshpande3/Machine-Learning-Notes.git
git@github.com:adeshpande3/MachineLearningReimplementations.git
git@github.com:adeshpande3/March-Madness-2017.git
git@github.com:adeshpande3/March-Madness-ML.git
git@github.com:adeshpande3/MLB.git
git@github.com:adeshpande3/MLB_Win_Predictor.git
git@github.com:adeshpande3/Music-Analysis.git
git@github.com:adeshpande3/NBA-Data-Visualization.git
git@github.com:adeshpande3/NLP-Stuff.git
git@github.com:adeshpande3/OpenAI_Gym_Universe.git
git@github.com:adeshpande3/Pandas-Tutorial.git
git@github.com:adeshpande3/PyTorch-Programs.git
git@github.com:adeshpande3/Quandl-Machine-Learning.git
git@github.com:adeshpande3/ReinforcementLearning.git
git@github.com:adeshpande3/Sensor-Collection-Display-App.git
git@github.com:adeshpande3/Tech-Plus-You-March-Madness.git
git@github.com:adeshpande3/Tensorflow-Programs-and-Tutorials.git
git@github.com:adeshpande3/Two-Sigma-Renting-Listing-Competition.git
git@github.com:adeshpande3/UCLA-Course-Notes.git
100  191k  100  191k    0     0   624k      0 --:--:-- --:--:-- --:--:--  622k
git@github.com:adeshpande3/WalmartLabs-ML-CodeSprint.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:adeshpande3/Accelerometer-Pogram.git
git@github.com:adeshpande3/adeshpande3.github.io.git
git@github.com:adeshpande3/Chatbot-Flask-Server.git
git@github.com:adeshpande3/Facebook-Messenger-Bot.git
git@github.com:adeshpande3/Generative-Adversarial-Networks.git
git@github.com:adeshpande3/Job-Descriptions-Hacker-Rank-Comp.git
git@github.com:adeshpande3/Kaggle-MNIST.git
git@github.com:adeshpande3/Kaggle-Zillow.git
git@github.com:adeshpande3/KaggleGhosts.git
git@github.com:adeshpande3/KaggleTitanic.git
git@github.com:adeshpande3/LSTM-Sentiment-Analysis.git
git@github.com:adeshpande3/Machine-Learning-Links-And-Lessons-Learned.git
git@github.com:adeshpande3/Machine-Learning-Notes.git
git@github.com:adeshpande3/MachineLearningReimplementations.git
git@github.com:adeshpande3/March-Madness-2017.git
git@github.com:adeshpande3/March-Madness-ML.git
git@github.com:adeshpande3/MLB.git
git@github.com:adeshpande3/MLB_Win_Predictor.git
git@github.com:adeshpande3/Music-Analysis.git
git@github.com:adeshpande3/NBA-Data-Visualization.git
git@github.com:adeshpande3/NLP-Stuff.git
git@github.com:adeshpande3/OpenAI_Gym_Universe.git
git@github.com:adeshpande3/Pandas-Tutorial.git
git@github.com:adeshpande3/PyTorch-Programs.git
git@github.com:adeshpande3/Quandl-Machine-Learning.git
git@github.com:adeshpande3/ReinforcementLearning.git
git@github.com:adeshpande3/Sensor-Collection-Display-App.git
git@github.com:adeshpande3/Tech-Plus-You-March-Madness.git
git@github.com:adeshpande3/Tensorflow-Programs-and-Tutorials.git
git@github.com:adeshpande3/Two-Sigma-Renting-Listing-Competition.git
git@github.com:adeshpande3/UCLA-Course-Notes.git
100  191k  100  191k    0     0   352k      0 --:--:-- --:--:-- --:--:--  352k
git@github.com:adeshpande3/WalmartLabs-ML-CodeSprint.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:diegoalejogm/AI-Nanodegree.git
git@github.com:diegoalejogm/artificial-intelligence-berkeley.git
git@github.com:diegoalejogm/Competitive-Programming.git
git@github.com:diegoalejogm/crts-transient-recognition.git
git@github.com:diegoalejogm/deep-q-learning.git
git@github.com:diegoalejogm/diegoalejogm.github.io.git
git@github.com:diegoalejogm/fbBumpy.git
git@github.com:diegoalejogm/gans.git
git@github.com:diegoalejogm/Google-Machine-Learning-Recipes.git
git@github.com:diegoalejogm/gym_gridworld.git
git@github.com:diegoalejogm/Hi.git
git@github.com:diegoalejogm/image-style-transfer.git
git@github.com:diegoalejogm/library.ml.git
git@github.com:diegoalejogm/MachineLearningCoursera.git
git@github.com:diegoalejogm/Mobile-Robotics-Projects.git
 64  147k   64 97203    0     0   314k      0 --:--:-- --:--:-- --:--:--  313kgit@github.com:diegoalejogm/myChatCatApp.git
git@github.com:diegoalejogm/node-startup.git
git@github.com:diegoalejogm/openai-k-armed-bandits.git
git@github.com:diegoalejogm/OpenMined.git
git@github.com:diegoalejogm/Paper-TR.git
git@github.com:diegoalejogm/PySyft.git
git@github.com:diegoalejogm/Reinforcement-Learning.git
git@github.com:diegoalejogm/Sonar.git
git@github.com:diegoalejogm/transient-recognition-experiments.git
100  147k  100  147k    0     0   486k      0 --:--:-- --:--:-- --:--:--  484k
git@github.com:diegoalejogm/udacity-ml-nanodegree.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0git@github.com:osh/async-rl.git
git@github.com:osh/async-rl-1.git
git@github.com:osh/author-classifier.git
git@github.com:osh/cae.git
git@github.com:osh/cluster.git
git@github.com:osh/cpp-ethereum.git
git@github.com:osh/czml.git
git@github.com:osh/DeepLearnToolbox.git
git@github.com:osh/deepmat.git
git@github.com:osh/dist_hyperas.git
git@github.com:osh/dogescript.git
git@github.com:osh/dqn.git
git@github.com:osh/fecapi.git
git@github.com:osh/gnuradio.git
git@github.com:osh/gnuradio.old.git
git@github.com:osh/gr-benchmark.git
git@github.com:osh/gr-bitcoin.git
git@github.com:osh/gr-chunky.git
git@github.com:osh/gr-etcetera.git
git@github.com:osh/gr-eventsim.git
git@github.com:osh/gr-eventstream.git
git@github.com:osh/gr-fec-test.git
git@github.com:osh/gr-fsk-burst.git
git@github.com:osh/gr-gps.git
git@github.com:osh/gr-ieee802-11.git
git@github.com:osh/gr-ieee802-15-4.git
git@github.com:osh/gr-latency.git
git@github.com:osh/gr-ldpc.git
git@github.com:osh/gr-lte.git
git@github.com:osh/gr-mediatools.git
git@github.com:osh/gr-modtool.git
git@github.com:osh/gr-pcap.git
git@github.com:osh/gr-pmtfile.git
git@github.com:osh/gr-psk-burst.git
git@github.com:osh/gr-pyqt.git
git@github.com:osh/gr-recipes.git
git@github.com:osh/gr-tf.git
git@github.com:osh/gr-theano.git
git@github.com:osh/gr-uhdgps.git
git@github.com:osh/grcompat.git
git@github.com:osh/grcon2016.git
git@github.com:osh/gsoc14-proposal.git
git@github.com:osh/hackrf.git
git@github.com:osh/inspectrum.git
git@github.com:osh/iris-classifier.git
git@github.com:osh/keras.git
git@github.com:osh/KerasGAN.git
git@github.com:osh/kerlym.git
git@github.com:osh/kml.git
git@github.com:osh/libtaxii.git
git@github.com:osh/liquid-dsp.git
git@github.com:osh/measurement_toolbox.git
git@github.com:osh/meta-sdr.git
git@github.com:osh/newsgroup-classifier.git
git@github.com:osh/nnplot.git
git@github.com:osh/nvidia-graphics-drivers.git
git@github.com:osh/openlte.git
git@github.com:osh/pgm_examples.git
git@github.com:osh/PyAV.git
git@github.com:osh/pybombs.git
git@github.com:osh/pykalman.git
git@github.com:osh/pylearn2.git
git@github.com:osh/rf_helicopter.git
git@github.com:osh/soltra_edge_tools.git
git@github.com:osh/system-bus-radio.git
git@github.com:osh/tensorflow.git
git@github.com:osh/Theano.git
git@github.com:osh/tpms.git
100  353k  100  353k    0     0   276k      0  0:00:01  0:00:01 --:--:--  276k
git@github.com:osh/volk_stats.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:tensorflow/adanet.git
git@github.com:tensorflow/addons.git
git@github.com:tensorflow/agents.git
git@github.com:tensorflow/autograph.git
git@github.com:tensorflow/benchmarks.git
git@github.com:tensorflow/cleverhans.git
git@github.com:tensorflow/community.git
git@github.com:tensorflow/compression.git
git@github.com:tensorflow/custom-op.git
git@github.com:tensorflow/data-validation.git
git@github.com:tensorflow/datasets.git
git@github.com:tensorflow/deepmath.git
git@github.com:tensorflow/docs.git
git@github.com:tensorflow/ecosystem.git
git@github.com:tensorflow/embedding-projector-standalone.git
git@github.com:tensorflow/estimator.git
git@github.com:tensorflow/examples.git
git@github.com:tensorflow/federated.git
git@github.com:tensorflow/fold.git
git@github.com:tensorflow/gan.git
git@github.com:tensorflow/graphics.git
git@github.com:tensorflow/haskell.git
git@github.com:tensorflow/hub.git
git@github.com:tensorflow/io.git
git@github.com:tensorflow/kfac.git
git@github.com:tensorflow/lattice.git
git@github.com:tensorflow/lingvo.git
git@github.com:tensorflow/lucid.git
git@github.com:tensorflow/magenta.git
git@github.com:tensorflow/magenta-demos.git
git@github.com:tensorflow/magenta-js.git
git@github.com:tensorflow/magenta-studio.git
git@github.com:tensorflow/mesh.git
git@github.com:tensorflow/metadata.git
git@github.com:tensorflow/minigo.git
git@github.com:tensorflow/mlir.git
git@github.com:tensorflow/model-analysis.git
git@github.com:tensorflow/model-optimization.git
git@github.com:tensorflow/models.git
git@github.com:tensorflow/moonlight.git
git@github.com:tensorflow/networking.git
git@github.com:tensorflow/neural-structured-learning.git
git@github.com:tensorflow/ngraph-bridge.git
git@github.com:tensorflow/nmt.git
git@github.com:tensorflow/playground.git
git@github.com:tensorflow/privacy.git
git@github.com:tensorflow/probability.git
git@github.com:tensorflow/profiler-ui.git
git@github.com:tensorflow/ranking.git
git@github.com:tensorflow/rust.git
git@github.com:tensorflow/serving.git
git@github.com:tensorflow/skflow.git
git@github.com:tensorflow/swift.git
git@github.com:tensorflow/swift-apis.git
git@github.com:tensorflow/swift-bindings.git
git@github.com:tensorflow/swift-models.git
git@github.com:tensorflow/tcav.git
git@github.com:tensorflow/tensor2tensor.git
git@github.com:tensorflow/tensorboard.git
git@github.com:tensorflow/tensorboard-plugin-example.git
git@github.com:tensorflow/tensorflow.git
git@github.com:tensorflow/tensorrt.git
git@github.com:tensorflow/tensorstore.git
git@github.com:tensorflow/text.git
git@github.com:tensorflow/tfjs.git
git@github.com:tensorflow/tfjs-converter.git
git@github.com:tensorflow/tfjs-core.git
git@github.com:tensorflow/tfjs-data.git
git@github.com:tensorflow/tfjs-examples.git
git@github.com:tensorflow/tfjs-layers.git
git@github.com:tensorflow/tfjs-models.git
git@github.com:tensorflow/tfjs-node.git
git@github.com:tensorflow/tfjs-tsne.git
git@github.com:tensorflow/tfjs-vis.git
git@github.com:tensorflow/tfjs-website.git
git@github.com:tensorflow/tfjs-wechat.git
git@github.com:tensorflow/tfrc.git
git@github.com:tensorflow/tfx.git
git@github.com:tensorflow/tpu.git
git@github.com:tensorflow/transform.git
100  452k  100  452k    0     0   623k      0 --:--:-- --:--:-- --:--:--  622k
git@github.com:tensorflow/workshops.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  522k    0   204    0     0    224      0  0:39:50 --:--:--  0:39:50   224git@github.com:r9y9/bandmat.git
git@github.com:r9y9/bayesian-kalmanfilter.git
git@github.com:r9y9/blog.git
git@github.com:r9y9/BNMF.jl.git
git@github.com:r9y9/caffe.git
git@github.com:r9y9/clif.git
git@github.com:r9y9/Colaboratory.git
git@github.com:r9y9/Compat.jl.git
git@github.com:r9y9/ConstantQ.jl.git
git@github.com:r9y9/Cxx.jl.git
git@github.com:r9y9/deepvoice3_pytorch.git
git@github.com:r9y9/demos.git
git@github.com:r9y9/demos-src.git
git@github.com:r9y9/docker-pytorch-apex.git
git@github.com:r9y9/DocOpt.jl.git
git@github.com:r9y9/Documenter.jl.git
git@github.com:r9y9/dotfiles.git
git@github.com:r9y9/DSP.jl.git
git@github.com:r9y9/espnet.git
git@github.com:r9y9/examples.git
git@github.com:r9y9/fabric.git
git@github.com:r9y9/fft.git
git@github.com:r9y9/FHMMs.jl.git
git@github.com:r9y9/gantts.git
git@github.com:r9y9/gentle.git
git@github.com:r9y9/go-cshared-examples.git
git@github.com:r9y9/go-dsp.git
git@github.com:r9y9/go-world.git
git@github.com:r9y9/go.matrix.git
git@github.com:r9y9/gossp.git
git@github.com:r9y9/HMMs.jl.git
git@github.com:r9y9/HTSEngineAPI.jl.git
git@github.com:r9y9/hts_engine_API.git
git@github.com:r9y9/hugo-zen.git
git@github.com:r9y9/install.git
git@github.com:r9y9/IterativeSolvers.jl.git
git@github.com:r9y9/julia.git
git@github.com:r9y9/julia-cxx.git
git@github.com:r9y9/julia-nmf-ss-toy.git
git@github.com:r9y9/JuliaTokyo3.git
git@github.com:r9y9/JuliaTokyo6.git
git@github.com:r9y9/jupyter-cxx-notebooks.git
git@github.com:r9y9/kaldi.git
git@github.com:r9y9/kaldi-io-for-python.git
git@github.com:r9y9/keras.git
git@github.com:r9y9/KinectGrabber.git
git@github.com:r9y9/legacy-homebrew.git
git@github.com:r9y9/Libfreenect2.jl.git
git@github.com:r9y9/LibOSXUVC.jl.git
git@github.com:r9y9/librosa.git
git@github.com:r9y9/LibSndFile.jl.git
git@github.com:r9y9/Logging.jl.git
git@github.com:r9y9/lws.git
git@github.com:r9y9/MCepAlpha.jl.git
git@github.com:r9y9/MeCab.jl.git
git@github.com:r9y9/media-player-demo.git
git@github.com:r9y9/MelGeneralizedCepstrums.jl.git
git@github.com:r9y9/merlin.git
git@github.com:r9y9/METADATA.jl.git
git@github.com:r9y9/midi.git
git@github.com:r9y9/MIDI.jl.git
git@github.com:r9y9/mnist.git
git@github.com:r9y9/Mocha.jl.git
git@github.com:r9y9/mpaligner.git
git@github.com:r9y9/naive_bayes.git
git@github.com:r9y9/neologdn.git
git@github.com:r9y9/nlp100.git
git@github.com:r9y9/nnet.git
git@github.com:r9y9/nnmnkwii.git
git@github.com:r9y9/nnmnkwii_gallery.git
git@github.com:r9y9/openFrameworks.git
git@github.com:r9y9/openFrameworks-template.git
git@github.com:r9y9/OpenNI2.git
git@github.com:r9y9/OpenNI2.jl.git
git@github.com:r9y9/OpenNMT-py.git
git@github.com:r9y9/open_jtalk.git
git@github.com:r9y9/pcdio_test.git
git@github.com:r9y9/pcl.git
git@github.com:r9y9/PCLLite.jl.git
git@github.com:r9y9/Polynomials.jl.git
git@github.com:r9y9/progressbar.git
git@github.com:r9y9/pykaldi.git
git@github.com:r9y9/pylibfreenect2.git
git@github.com:r9y9/pyopenjtalk.git
git@github.com:r9y9/pypcl.git
git@github.com:r9y9/pyreaper.git
git@github.com:r9y9/pysptk.git
git@github.com:r9y9/python-neural-net-toy-codes.git
git@github.com:r9y9/Python-Wrapper-for-World-Vocoder.git
git@github.com:r9y9/pytorch.git
git@github.com:r9y9/r9y9.github.io.git
git@github.com:r9y9/REAPER.git
git@github.com:r9y9/REAPER.jl.git
git@github.com:r9y9/RobustPCA.jl.git
git@github.com:r9y9/robust_pca.git
git@github.com:r9y9/segmentation-kit.git
git@github.com:r9y9/setup.git
git@github.com:r9y9/SimpleBinaryManagementExample.jl.git
git@github.com:r9y9/SpeechBase.jl.git
100  522k  100  522k    0     0   539k      0 --:--:-- --:--:-- --:--:--  538k
git@github.com:r9y9/SPTK.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:jayleicn/animeGAN.git
git@github.com:jayleicn/CatPapers.git
git@github.com:jayleicn/chrome_extensions_and_apps_programming.git
git@github.com:jayleicn/classification-with-coarse-fine-labels.git
git@github.com:jayleicn/dotfiles.git
git@github.com:jayleicn/examples.git
git@github.com:jayleicn/ext_matconvnet.git
git@github.com:jayleicn/hashing-baseline-for-image-retrieval.git
git@github.com:jayleicn/hosts.git
git@github.com:jayleicn/ImageNet-Training.git
git@github.com:jayleicn/jayleicn.github.io.git
git@github.com:jayleicn/kinetics-i3d.git
git@github.com:jayleicn/maskrcnn-benchmark.git
git@github.com:jayleicn/MAttNet.git
git@github.com:jayleicn/my-scripts.git
git@github.com:jayleicn/nn.git
git@github.com:jayleicn/QANet-pytorch.git
git@github.com:jayleicn/scipy-lecture-notes-zh-CN.git
git@github.com:jayleicn/simple-amt.git
git@github.com:jayleicn/Smart-System.git
git@github.com:jayleicn/temporal-segment-networks.git
git@github.com:jayleicn/temporal-shift-module.git
git@github.com:jayleicn/transfer-DPSH.git
git@github.com:jayleicn/TVQA.git
git@github.com:jayleicn/TVQAplus.git
100  146k  100  146k    0     0   389k      0 --:--:-- --:--:-- --:--:--  389k
git@github.com:jayleicn/video-nonlocal-net.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0git@github.com:Zardinality/BEGAN.git
git@github.com:Zardinality/cs231n_project.git
git@github.com:Zardinality/EC_VU_2014.git
git@github.com:Zardinality/GTM_Cover_Crawler.git
git@github.com:Zardinality/models.git
git@github.com:Zardinality/mx-rfcn.git
git@github.com:Zardinality/mxnet.git
git@github.com:Zardinality/online-cv.git
git@github.com:Zardinality/poisson-matting.git
git@github.com:Zardinality/pytorch-flows.git
git@github.com:Zardinality/SeamCarver.git
git@github.com:Zardinality/tensorflow.git
git@github.com:Zardinality/TF-deformable-conv.git
git@github.com:Zardinality/TF_Deformable_Net.git
git@github.com:Zardinality/Tiled-Matrix-Multiplication-in-Cuda.git
git@github.com:Zardinality/unrolled-GAN.git
git@github.com:Zardinality/WGAN-tensorflow.git
100  102k  100  102k    0     0   364k      0 --:--:-- --:--:-- --:--:--  364k
git@github.com:Zardinality/zardinality.github.io.git
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  199k    0   364    0     0    601      0  0:05:39 --:--:--  0:05:39   600git@github.com:timsainb/AVGN.git
git@github.com:timsainb/birdbrain.git
git@github.com:timsainb/courses.git
git@github.com:timsainb/curriculum_vitae.git
git@github.com:timsainb/cv.git
git@github.com:timsainb/d3-plugins.git
git@github.com:timsainb/datasciencecoursera.git
git@github.com:timsainb/datasharing.git
git@github.com:timsainb/example_r_colab_project.git
git@github.com:timsainb/ExData_Plotting1.git
git@github.com:timsainb/GAIA.git
git@github.com:timsainb/GCD-Course-Project.git
git@github.com:timsainb/glab_oe_rig_tools.git
git@github.com:timsainb/graph_research_notes.git
git@github.com:timsainb/K3D-jupyter.git
git@github.com:timsainb/LeabraExample.git
git@github.com:timsainb/morphs.git
git@github.com:timsainb/Motif_VAE.git
git@github.com:timsainb/mountainlab_pytools.git
git@github.com:timsainb/noisereduce.git
git@github.com:timsainb/ParallelsBirdsongLanguagePaper.git
git@github.com:timsainb/plugin-GUI.git
git@github.com:timsainb/ProgrammingAssignment2.git
git@github.com:timsainb/pyoperant.git
git@github.com:timsainb/python_spectrograms_and_inversion.git
git@github.com:timsainb/RepData_PeerAssessment1.git
git@github.com:timsainb/repulsive-autoencoder.git
git@github.com:timsainb/science_vector_images.git
git@github.com:timsainb/StatisticalWordLearningLeabra.git
git@github.com:timsainb/Tensorflow-MultiGPU-VAE-GAN.git
git@github.com:timsainb/tensorflow2-generative-models.git
git@github.com:timsainb/timsainb.github.io.git
git@github.com:timsainb/timsainburg.com.git
git@github.com:timsainb/Twitch-plays-LSD-neural-net.git
100  199k  100  199k    0     0   309k      0 --:--:-- --:--:-- --:--:--  309k
git@github.com:timsainb/visualize_wordpress_posts.git

In [0]:

!pip install tensorflow tensorflow_probability tensorflow_datasets